In [1]:
import re, sys, os
from json import dump
from tqdm import tqdm
import time
from collections import defaultdict
import requests
import grequests
from itertools import islice
# user packages
from bs4 import BeautifulSoup
# from urllib.request import urlopen, Request
import json
sys.path.append('../')
from scripts.oldlistings import get_oldlistings_page
import pickle
from scripts.env import scraper_api_key # replave with your own api key
base_url = 'https://www.oldlistings.com.au'



/home/even/repo/project-2-group-real-estate-industry-project-2/.venv/lib/python3.10/site-packages/grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/home/even/repo/project-2-group-real-estate-industry-project-2/.venv/lib/python3.10/site-packages/urllib3/util/__init__.py)', 'urllib3.util.ssl_ (/home/even/repo/project-2-group-real-estate-industry-project-2/.venv/lib/python3.10/site-packages/urllib3/util/ssl_.py)']. 
  curious_george.patch_all(thread=False, select=False)


In [2]:
# this is for testing using downloaded html
# please use normal url for production

# read html with bs4
path = "../data/landing/South Melbourne Rent Real Estate Old Listings.html"

with open(path, 'r') as f:
    webpage = f.read()

soup = BeautifulSoup(webpage)

### function to get info from a property

In [24]:
def get_info(p):
    # try to get data in json format 
    info_dict = defaultdict(None)
    info_dict['lat'] = p.get('data-lat')
    info_dict['lng'] = p.get('data-lng')
    info_dict['rented_prices'] = []

    p = p.find('section', {'class':"grid-100 grid-parent"}) # replace p with its only (useful)child

    title = p.find('section', {'class':"grid-65 tablet-grid-65 clearfix"})
    ad = p.find('section', {'class':"grid-35 tablet-grid-35 price"})
    list_of_history = p.find('section', {'class':"grid-100 historical-price"})


    # title
    info_dict['address'] = title.find('h2', {'class': 'address'}).text
    # other metadata: bed, bath, car etc
    # this do not garentee any structure of the data
    info_dict['meta_data'] = []
    for meta_data in title.find_all('p', {'class': re.compile("property-meta")}):
        obj = {}
        obj['label'] = meta_data.get('class')[1]
        obj['description'] = meta_data.find('span').text.split(' ')[0]
        obj['quantity'] = meta_data.contents[1].strip()
        info_dict['meta_data'].append(obj)


    # ad : we dont use ad

    # list of history
    for line in list_of_history.find('ul').find_all('li'):
        record = {}
        record['date'] = line.find('span').text
        record['price'] = line.contents[1]
        info_dict['rented_prices'].append(record)

    return info_dict

In [25]:
# soup.find_all('div', {'class':"content-col"})
def scrape_page(soup):
    properties = soup.find('div', {'class':"content-col"}).findChildren("div" , recursive=False)
    list_of_properties = []
    for property in properties:
        list_of_properties.append(get_info(property))
        # get_info(property)
    return list_of_properties

In [6]:
# testing json output, for preprocessing use

# json.dump(get_oldlistings_page(soup), open('test.json', 'w'))

# Sitemap
page=1 is the 2nd page
no page tag is the first page

## Getting the sitemap

https://www.oldlistings.com.au/site-map?page=1&state=VIC

In [1]:
l = ["https://www.oldlistings.com.au/site-map?state=VIC"]
for page in range(1,18):
    l.append(f"https://www.oldlistings.com.au/site-map?page={page}&state=VIC")
l

['https://www.oldlistings.com.au/site-map?state=VIC',
 'https://www.oldlistings.com.au/site-map?page=1&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=2&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=3&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=4&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=5&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=6&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=7&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=8&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=9&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=10&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=11&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=12&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=13&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=14&state=VIC',
 'https://www.oldlistings.com.au/site-map?page=15&state=VIC',
 'https://www.oldlistings

In [5]:
# save data in a csv, each entries is a new row
with open("../data/landing/oldlistings_sitemap.csv", "w") as f:
    for url in l:
        f.write(url)
        f.write("\n")

In [12]:
for i in range(0,18):
    payload = { 'api_key': scraper_api_key, 'url': l[i] }
    r = requests.get('https://api.scraperapi.com/', params=payload)
    with open(f"../data/landing/oldlistings/sitemap/{i}.html", "w") as f:
        f.write(r.text)

## get urls from sitemap

In [40]:
path = "../data/landing/oldlistings/sitemap/0.html"
bs4 = BeautifulSoup(open(path, 'r').read(),"html.parser")

In [ ]:
# end with buy/
buy_re = re.compile(r".*buy\/$")
rent_re = re.compile(r".*rent\/$")
#bs4.find_all('table', {'class':"sticky-enabled"})[0].find_all('tr')[1]
links = bs4.find_all('table', {'class':"sticky-enabled"})[0].find_all('a', href=buy_re, recursive=True)
for link in links:
    print(link['href'])

In [8]:
buy_re = re.compile(r".*buy\/$")
rent_re = re.compile(r".*rent\/$")
list_buy = []
list_rent = []
for i in range(0,18):
    path = f"../data/landing/oldlistings/sitemap/{i}.html"
    bs4 = BeautifulSoup(open(path, 'r').read(),"html.parser")
    # buy part
    links = bs4.find_all('table', {'class':"sticky-enabled"})[0].find_all('a', href=buy_re, recursive=True)
    for _link in links:
        list_buy.append(_link['href'])
    # rent part
    links = bs4.find_all('table', {'class':"sticky-enabled"})[0].find_all('a', href=rent_re, recursive=True)
    for _link in links:
        list_rent.append(_link['href'])
list_buy = list(set(list_buy))
list_rent = list(set(list_rent))

In [9]:
len(list_buy), len(list_rent)

(3504, 3504)

In [12]:
with open("../data/landing/oldlistings/buy_area_pythonobj", "wb") as f:
    pickle.dump(list_buy, f)
with open("../data/landing/oldlistings/rent_area_pythonobj", "wb") as f:
    pickle.dump(list_rent, f)

# Buy: First Page scraping
first page includes href to all pages

In [23]:
list_buy[1].split('/')[-2]

'buy'

In [15]:
progress = 0

In [24]:
for _link in list_buy:
    payload = { 'api_key': scraper_api_key, 'url': f"{base_url}{_link}" }
    _items = _link.split('/')
    r = requests.get('https://api.scraperapi.com/', params=payload)
    with open(f"../data/landing/oldlistings/buy_html/{_items[-3]}_{_items[-4]}.html", "w") as f:
        f.write(r.text)

# Rent: first page scraping
concurrent requests

In [6]:
# Function to execute requests in batches of 4
def fetch_in_batches(urls, batch_size=4):
    print(f"Fetching {len(urls)/batch_size} batches")
    for batch in tqdm(iter_batches(urls, batch_size)):
        rs = (grequests.get('https://api.scraperapi.com/', params = { 'api_key': scraper_api_key, 'url': url }) for url in batch)
        responses = grequests.map(rs)
        for i in range(len(responses)):
            _items = batch[i].split('/')
            r = responses[i]
            with open(f"../data/landing/oldlistings/rent_html/{_items[-3]}_{_items[-4]}.html", "w") as f:
                f.write(r.text)

# Helper function to split URLs into batches of a given size
def iter_batches(iterable, batch_size):
    it = iter(iterable)
    while True:
        batch = list(islice(it, batch_size))
        if not batch:
            break
        yield batch

In [ ]:
for i in range(len(list_rent)):
    list_rent[i] = base_url + list_rent[i]

In [ ]:
fetch_in_batches(list_rent, batch_size=4)

# all page links
except first page

In [2]:
def find_pages(soup,list_out):
    _temp =  soup.find_all('ul',{'class': 'pagination'} )
    if len(_temp) > 0:
        # use first one which is the page list
        _temp = _temp[0].find_all('li', {'class': False})
        for _t in _temp:
            list_out.append(_t.find('a', href=True)['href'])
    

In [3]:
def find_directory(dir):
    # reset list
    list_out = []
    # run loop
    for file_path in tqdm(os.listdir(dir)):
        full_p = dir + file_path
        bs4 = BeautifulSoup(open(full_p, 'r').read(),"html.parser")
        find_pages(bs4,list_out)
    return list_out

## Rent

In [4]:
# not first pages
base_rent_dir = "../data/landing/oldlistings/rent_html/"
list_rent_2 = find_directory(base_rent_dir)


100%|██████████| 3504/3504 [00:39<00:00, 89.57it/s] 


In [12]:
list_rent_2 = list(set(list_rent_2))
pickle.dump(list_rent_2, open("../data/landing/oldlistings/rent_pages.p", "wb"))

## Buy

In [6]:
# not first pages
base_buy_dir = "../data/landing/oldlistings/buy_html/"
list_buy_2 = find_directory(base_buy_dir)


100%|██████████| 3504/3504 [00:56<00:00, 62.13it/s]


In [13]:
list_buy_2 = list(set(list_buy_2))
pickle.dump(list_buy_2, open("../data/landing/oldlistings/buy_pages.p", "wb"))

# Scrape all links (moved to other notebook)
due to large amount, scrape by small batches

In [1]:
import pickle
base_url = 'https://www.oldlistings.com.au'
# import requests
import grequests
from itertools import islice
from tqdm import tqdm

In [2]:
with open("../data/landing/oldlistings/rent_pages.p", "rb") as f:
    list_rent_2 = pickle.load(f)
with open("../data/landing/oldlistings/buy_pages.p", "rb") as f:
    list_buy_2 = pickle.load(f)

In [3]:
len(list_rent_2), len(list_buy_2)

(25776, 36227)

In [4]:
list_rent_2[0]

'/real-estate/VIC/Bentleigh+East/3165/rent/110'

In [5]:
def fetch_in_batches_2(urls, batch_size=5, type = 'rent', base_url = 'https://www.oldlistings.com.au'):
    print(f"Fetching {len(urls)/batch_size} batches")
    for batch in tqdm(iter_batches(urls, batch_size)):
        rs = (grequests.get('https://api.scraperapi.com/', params = { 'api_key': scraper_api_key, 'url': base_url + url }) for url in batch)
        responses = grequests.map(rs)
        for i in range(len(responses)):
            _items = batch[i].split('/')
            r = responses[i]
            print(r)
            with open(f"../data/landing/oldlistings/{type}_html/{_items[-3]}_{_items[-4]}_{_items[-1]}.html", "w") as f:
                f.write(r.text)